In [ ]:
# File: roller.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
from frontend import App

# create an app
app = App.create("roller")

# create a cylinder mesh for rollers
r, half_l = 0.15, 0.7
V, F = app.mesh.cylinder(r=r, min_x=-half_l, max_x=half_l, n=40)
app.asset.add.tri("cylinder", V, F)

# create a knot tetrahedral mesh
V, F, T = app.mesh.preset("knot").tetrahedralize().normalize()
app.asset.add.tet("knot", V, F, T)

# create a scene
scene = app.scene.create()

# add knot object with friction
obj = scene.add("knot").scale(0.6).at(0, 0.4, 0).jitter().rotate(270, "y")
obj.param.set("friction", 0.5)

# add invisible floor
scene.add.invisible.wall([0, -1, 0], [0, 1, 0])

# create two pairs of counter-rotating rollers to flatten the knot
half_gap, left, right = 1.1 * r, [], []
left.append(scene.add("cylinder").rotate(90, "y").at(-half_gap, 0, 0))
left.append(scene.add("cylinder").rotate(90, "y").at(-half_gap, -2 * half_gap, 0))
right.append(scene.add("cylinder").rotate(90, "y").at(half_gap, 0, 0))
right.append(scene.add("cylinder").rotate(90, "y").at(half_gap, -2 * half_gap, 0))

# set spinning motion for rollers
w = 360.0
for obj in left:
    obj.pin().spin(axis=[0, 0, -1], angular_velocity=w)
for obj in right:
    obj.pin().spin(axis=[0, 0, 1], angular_velocity=w)

# set directional coloring for all rollers
for obj in left + right:
    obj.direction_color(1, 0, 0)

# set preview options
opts = {
    "pin": False,
    "wireframe": True,
    "lookat": [0, -0.25, 0],
    "eyeup": 0.25,
    "fov": 55,
}

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview(options=opts)

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters
session.param.set("frames", 180)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview(options=opts)

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate(options=opts)

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()